## Importing, cleaning, and wrangling data

In [2]:
import pandas as pd
import numpy as np
from pandas.api.types import CategoricalDtype
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.mosaicplot import mosaic

In [69]:
# import data with a context manager
with open("data_ks.csv") as fp:
    df = pd.read_csv(fp) 
    names = df['token'].tolist()

# convert to categorical data in the df
catcol = ["age", "gender", "platform", "income", "experience", "rides_per_week", "num_bikes_owned", "bike_cost", "location", "environment", 'accessory_factor' ]
for col in catcol:
    df[col] = df[col].astype('category')

print(df.info())
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1691 entries, 0 to 1690
Data columns (total 25 columns):
age                        1691 non-null category
gender                     1691 non-null category
platform                   1691 non-null category
income                     1691 non-null category
experience                 1691 non-null category
rides_per_week             1691 non-null category
bike_type                  1691 non-null object
num_bikes_owned            1691 non-null category
bike_cost                  1691 non-null category
location                   1691 non-null category
environment                1691 non-null category
companions                 1691 non-null object
conditions                 1691 non-null object
activities                 1691 non-null object
reasons_to_bike            1691 non-null object
reasons_to_not_bike        1691 non-null object
reason_for_smarthalo       1691 non-null object
has_smarthalo_1            1691 non-null object
accessory

(1691, 25)

In [70]:
for i in catcol:
    print(df[i].value_counts(dropna=False), '\n')

25-34 years    481
35-44 years    472
45-54 years    289
55+ years      285
0-25 years     108
45-45 years     56
Name: age, dtype: int64 

Male                 1431
Female                244
Prefer not to say       9
Non-binary              7
Name: gender, dtype: int64 

iOS        962
Android    729
Name: platform, dtype: int64 

40,000 - 79,000 / year    573
80,000+ / year            564
0 - 39,999 / year         301
Prefer not to say         253
Name: income, dtype: int64 

8     473
7     463
6     248
9     186
5     121
10    106
4      50
3      23
2      12
1       9
Name: experience, dtype: int64 

3 or more times a week    942
1-2 times a week          520
Less than once a week     229
Name: rides_per_week, dtype: int64 

1       682
2       588
3+      410
None     11
Name: num_bikes_owned, dtype: int64 

1000 - 2000      554
500 - 1000       490
2000 - 4000      309
100 - 500        204
4000+            122
Less than 100     12
Name: bike_cost, dtype: int64 

United States

## Segmentation/Filtering

This section creates a filtered set of data that we will compare against the baseline.

### Filter Criteria

This is where we set the filter criteria that will be used in all the output sections below

In [96]:
#this block is used to segment by categorical dimensions
#segment_text = 'United States'
#segment_df = df[df.location==segment_text]
#segment_text = '45-54 years'
#segment_df = df[df.age==segment_text]

# this block is used to segment by multi-select dimensions
#segment_text = 'Transportation'
#segment_df = df[df['activities'].str.contains(segment_text)]
segment_text = 'don\'t have to look at my phone '
segment_df = df[df['reason_for_smarthalo'].str.contains(segment_text)]

segment_df.shape

#POST: segment_df contains the filtered data we will be outputting below
#POST: segment_text contains outputable text that will be used below

(1158, 25)

## Categorical Data Output

In [97]:
#vars for output
precision = 2
output_data = []

for category in catcol:
    all_values = df[category].value_counts(normalize=True, sort=False)
    segment_values = segment_df[category].value_counts(normalize=True, sort=False)

    for i in range(0, len(all_values)):
        label = "{}:{}".format(category,all_values.index[i]) 
        cat_diff = round((segment_values[i]-all_values[i])/all_values[i]*100, precision)
        formatted_all = round(all_values[i]*100, precision)
        formatted_segment = round(segment_values[i]*100, precision)
        
        output_data.append([label, formatted_all, formatted_segment, cat_diff])


segment_category_df = pd.DataFrame(output_data, columns=['category', '% All', "% {}".format(segment_text), '% difference'])
#segment_category_df.to_csv('segment_category_output.csv', index = False)
print(segment_category_df.to_string())

                                              category  % All  % don't have to look at my phone   % difference
0                                       age:0-25 years   6.39                               6.65          4.11
1                                      age:25-34 years  28.44                              31.17          9.60
2                                      age:35-44 years  27.91                              28.41          1.79
3                                      age:45-45 years   3.31                               3.11         -6.13
4                                      age:45-54 years  17.09                              15.03        -12.08
5                                        age:55+ years  16.85                              15.63         -7.26
6                                        gender:Female  14.43                              14.59          1.14
7                                          gender:Male  84.62                              84.37         -0.30
8

## Multi-Select Data Output

In [98]:
#output vars
segment_count = segment_df.shape[0]
all_count = df.shape[0]
output_data = []

def output_multi_select_counts(option_list, segment_df, all_df, column_name):
    output_data = []
    segment_row_count = segment_df.shape[0]
    all_row_count = all_df.shape[0]
    
    for x in range(0, len(option_list)):
        option = option_list[x]
        segment_count = 0
        segment_percent = 0
        all_count = 0
        all_percent = 0
        difference_percent = 0
        
        #if we have the values, fill them in, otherwise use the zeroes above
        try:
            all_count = all_df[column_name].str.contains(option).value_counts()[True]
            all_percent = round(all_count/all_row_count*100, 2)
            segment_count = segment_df[column_name].str.contains(option).value_counts()[True]
            segment_percent = round(segment_count/segment_row_count*100, 2)
            difference_percent = round((segment_percent-all_percent)/all_percent*100,2)
        except KeyError: 
            pass
        
        output_data.append([option, all_count, segment_count, all_percent, segment_percent, difference_percent])
    
    segment_multi_df = pd.DataFrame(output_data, columns=[column_name, 'all count', 'segment count', "all %", "segment %", "difference %"])
    #segment_multi_df.to_csv('segment_multi_{}_output.csv'.format(column_name), index = False)
    print(segment_multi_df.to_string())
    

'''
The following values are just grabbed from the surveys.  I could probably use a CountVectorizer to do it in a more scalable way
'''

bike_types = ['Road bike', 'Mountain bike', 'Hybrid bike', 'Bike share bike', 'E-bike', 'Cruiser / City bike', 'Cargo bike', 'Single-speed bike', 'Gravel bike']
companions = ['I bike alone', 'With family and friends', 'With young children', 'With a riding pack']
conditions = ['Temperate', 'Hot/Humid', 'Rain', 'Snow/Ice', 'Night']
activities = ['Commuting', 'Chores / Errands', 'Work', 'Transportation', 'Mountain biking', 'Touring', 'Racing', 'Fitness', 'BMX', 'Pleasure rides']
reasons_to_bike = ['convenient', 'reliable', 'faster', 'cheaper', 'environment', 'healthy', 'friends/family', 'hobby/interest', 'identity/culture', 'more fun']
reasons_to_not_bike = ['weather', 'long/steep', 'don\'t know the way', 'have to carry things', 'don\'t want to sweat', 'not safe for me', 'not safe for my bike', 'feel lazy', 'wouldn\'t be fun', 'too dark']
reason_for_smarthalo = ['navigate', 'keep track of my fitness goals', 'keep track of my environmental goals', 'stay safe', 'keep my bike safe', 'don\'t have to look at my phone', 'it\'s cool!']
accessory_channels = ['Online forums', 'Social media', 'Amazon/e-commerce', 'Chats with friends or family', 'Retail stores']

# output

print("Segment: {}\n===============\n\n".format(segment_text))

output_multi_select_counts(bike_types, segment_df, df, "bike_type")
print("\n")
output_multi_select_counts(companions, segment_df, df, "companions")
print("\n")
output_multi_select_counts(conditions, segment_df, df, "conditions")
print("\n")
output_multi_select_counts(activities, segment_df, df, "activities")
print("\n")
output_multi_select_counts(reasons_to_bike, segment_df, df, "reasons_to_bike")
print("\n")
output_multi_select_counts(reasons_to_not_bike, segment_df, df, "reasons_to_not_bike")
print("\n")
output_multi_select_counts(reason_for_smarthalo, segment_df, df, "reason_for_smarthalo")
print("\n")
output_multi_select_counts(accessory_channels, segment_df, df, "accessory_channels")


Segment: don't have to look at my phone 


             bike_type  all count  segment count  all %  segment %  difference %
0            Road bike        743            510  43.94      44.04          0.23
1        Mountain bike        495            323  29.27      27.89         -4.71
2          Hybrid bike        413            292  24.42      25.22          3.28
3      Bike share bike        130             99   7.69       8.55         11.18
4               E-bike        261            169  15.43      14.59         -5.44
5  Cruiser / City bike        402            276  23.77      23.83          0.25
6           Cargo bike         61             40   3.61       3.45         -4.43
7    Single-speed bike        158            112   9.34       9.67          3.53
8          Gravel bike        189            136  11.18      11.74          5.01


                companions  all count  segment count  all %  segment %  difference %
0             I bike alone       1427            987  84.39 